In [ ]:
import sys
sys.path.append('../utils/analysis/')

import pandas as pd
import polars as pl
from filtering import Filter

In [2]:
file = "../physionet.org/files/mimiciv/3.1/icu/chartevents.csv.gz"

In [3]:
filter = Filter(file, "chartevents", debug=True)
df = filter.search_subject(10000690)

[Filter] Initialized for chartevents with 313645063 rows, sorted by subject_id
[search_subject] Searching for subject_id: 10000690
[DEBUG] 13648534 too high at index 156822531, searching left. Indices: low=0, high=313645062
[DEBUG] 11804901 too high at index 78411265, searching left. Indices: low=0, high=156822530
[DEBUG] 10918410 too high at index 39205632, searching left. Indices: low=0, high=78411264
[DEBUG] 10461415 too high at index 19602815, searching left. Indices: low=0, high=39205631
[DEBUG] 10236332 too high at index 9801407, searching left. Indices: low=0, high=19602814
[DEBUG] 10124291 too high at index 4900703, searching left. Indices: low=0, high=9801406
[DEBUG] 10051825 too high at index 2450351, searching left. Indices: low=0, high=4900702
[DEBUG] 10023117 too high at index 1225175, searching left. Indices: low=0, high=2450350
[DEBUG] 10012055 too high at index 612587, searching left. Indices: low=0, high=1225174
[DEBUG] 10005817 too high at index 306293, searching left

In [6]:
df.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000690,25860671,37081114,3527.0,2150-11-04 12:00:00,2150-11-04 12:05:00,223986,Rhonchi,NaN,NaN,0.0
1,10000690,25860671,37081114,3527.0,2150-11-04 12:00:00,2150-11-04 12:05:00,223988,Rhonchi,NaN,NaN,0.0
2,10000690,25860671,37081114,3527.0,2150-11-04 12:00:00,2150-11-04 12:05:00,224369,Thick,NaN,NaN,0.0
3,10000690,25860671,37081114,3527.0,2150-11-04 12:00:00,2150-11-04 12:05:00,224370,Bloody,NaN,NaN,0.0
4,10000690,25860671,37081114,3527.0,2150-11-04 12:00:00,2150-11-04 12:05:00,224372,Nasotrachial Suction,NaN,NaN,0.0


In [7]:
# Read the CSV file and get unique subject_ids
unique_subject_ids = pd.read_csv(file, usecols=['subject_id'])['subject_id'].unique()

# Convert to DataFrame and save to CSV
unique_df = pd.DataFrame(unique_subject_ids, columns=['subject_id'])
unique_df.to_csv('unique_subject_ids.csv', index=False)